데이터가 Sparse matrix 일 때 cosine distance 를 이용하면 대부분의 거리가 0.9 ~ 1.0 에 가깝습니다. 그렇기 때문에 bag of words model 을 이용하는 term frequency matrix 에서는 k-means++ 가 큰 의미가 없습니다. 

k-means++ 은 distance distribution 이 한쪽에 치우치지 않았을 때 잘 작동합니다. Similar cut initializer 는 term frequency matrix 와 같은 sparse vector 에서 효율적으로 initial points 를 선택하기 위한 방법입니다. 더 자세한 설명은 아래의 블로그를 참고해주세요.

https://lovit.github.io/nlp/machine%20learning/2018/03/19/kmeans_initializer/

In [1]:
from lovit_textmining_dataset.navernews_10days import get_bow

x, idx_to_vocab, vocab_to_idx = get_bow(date='2016-10-20', tokenize='noun')
x.shape

(30091, 9774)

idx2vocab 은 9.774 개의 단어들의 idx 에 해당하는 단어가 저장되어 있습니다. 이는 cluster labeling 에 이용됩니다.

In [2]:
print(idx_to_vocab[5535:5540])

['아이스크림', '아이엠', '아이오아이', '아이콘', '아이템']


In [3]:
import time
import sys
sys.path.append('../')
import soyclustering

print(soyclustering.__version__)

n_clusters = 500

0.1.0


## Time comparison

### Initializer: similar cut vs. k-means++

k-means++ 을 이용할 경우 (229253, 96136) 데이터에 대하여 initialization 에 856.01 초를 이용합니다.

similar cut 은 비슷한 점들을 initial points 로 선택하지 않으면서도 0.427 초만에 initialization 이 가능합니다. distance distribution 이 0.9 ~ 1.0 에 치우친 경우에는 k-means++ 은 잘 작동하지 않습니다.

In [4]:
import numpy as np
from soyclustering import SphericalKMeans

np.random.seed(0)

kmeans = SphericalKMeans(
    n_clusters=n_clusters,
    init='k-means++',
    sparsity=None,
    max_iter=10,
    tol=0.0001,
    verbose = True
)

t = time.time()
labels = kmeans.fit_predict(x)
t = time.time() - t

print('process time = {} seconds'.format('%.3f' % t))

initialization_time=15.966800 sec, sparsity=0.00824
n_iter=1, changed=29966, inertia=15058.407, iter_time=2.329 sec, sparsity=0.115
n_iter=2, changed=5083, inertia=11143.195, iter_time=2.230 sec, sparsity=0.108
n_iter=3, changed=1971, inertia=10686.542, iter_time=2.212 sec, sparsity=0.106
n_iter=4, changed=927, inertia=10553.024, iter_time=2.202 sec, sparsity=0.106
n_iter=5, changed=474, inertia=10506.710, iter_time=2.265 sec, sparsity=0.105
n_iter=6, changed=281, inertia=10482.177, iter_time=2.238 sec, sparsity=0.105
n_iter=7, changed=172, inertia=10467.970, iter_time=2.269 sec, sparsity=0.105
n_iter=8, changed=131, inertia=10458.836, iter_time=2.223 sec, sparsity=0.105
n_iter=9, changed=144, inertia=10448.147, iter_time=2.185 sec, sparsity=0.105
n_iter=10, changed=148, inertia=10438.499, iter_time=2.216 sec, sparsity=0.105
process time = 38.570 seconds


In [5]:
from soyclustering import SphericalKMeans

np.random.seed(0)

kmeans = SphericalKMeans(
    n_clusters=n_clusters,
    init='similar_cut',
    sparsity=None,
    max_iter=10,
    tol=0.0001,
    verbose = True
)

t = time.time()
labels = kmeans.fit_predict(x)
t = time.time() - t

print('process time = {} seconds'.format('%.3f' % t))

initialization_time=0.072077 sec, sparsity=0.00677
n_iter=1, changed=29953, inertia=15067.618, iter_time=2.316 sec, sparsity=0.113
n_iter=2, changed=6603, inertia=11585.212, iter_time=2.235 sec, sparsity=0.104
n_iter=3, changed=2989, inertia=11031.648, iter_time=2.265 sec, sparsity=0.102
n_iter=4, changed=1509, inertia=10799.084, iter_time=2.238 sec, sparsity=0.101
n_iter=5, changed=710, inertia=10705.900, iter_time=2.219 sec, sparsity=0.1
n_iter=6, changed=509, inertia=10668.506, iter_time=2.243 sec, sparsity=0.1
n_iter=7, changed=403, inertia=10637.684, iter_time=2.225 sec, sparsity=0.0999
n_iter=8, changed=347, inertia=10613.087, iter_time=2.215 sec, sparsity=0.0998
n_iter=9, changed=286, inertia=10589.020, iter_time=2.226 sec, sparsity=0.0997
n_iter=10, changed=163, inertia=10575.444, iter_time=2.258 sec, sparsity=0.0997
process time = 22.660 seconds


### soyclustering vs. scikit-learn

각 데이터와 centroids 와의 거리 계산 후, 새로운 cluster 에 데이터를 할당하는 부분이 scikit-learn 과 다르게 구현되어 있습니다. 현재 (versio 0.0.1)는 scikit-learn 의 k-means 보다 약 1.1 배 정도 더 긴 계산시간이 필요합니다.

scikit-learn 은 Euclidean distance 를 이용합니다. L2 normalized 된 centroids 가 아닌 L1 normalized 이기 때문에 Cosine 과 Euclidean 의 값은 차이가 있습니다.

In [6]:
import sklearn
print(sklearn.__version__)

# scikit-learn KMeans use only Euclidean
from sklearn.preprocessing import normalize
x_ = normalize(x)

0.22.2.post1


In [7]:
from sklearn.cluster import KMeans

kmeans_sklearn = KMeans(
    n_clusters=n_clusters,
    init='k-means++',
    n_init=1,
    max_iter=10,
    tol=0.0001,
    verbose=1
)

t = time.time()
_label = kmeans_sklearn.fit_predict(x_)
t = time.time() - t

print('process time = {} seconds'.format('%.3f' % t))

Initialization complete
Iteration  0, inertia 19437.835
Iteration  1, inertia 16233.231
Iteration  2, inertia 15896.028
Iteration  3, inertia 15745.917
Iteration  4, inertia 15653.595
Iteration  5, inertia 15604.705
Iteration  6, inertia 15580.258
Iteration  7, inertia 15567.442
Iteration  8, inertia 15559.196
Iteration  9, inertia 15553.098
process time = 92.501 seconds


## cluster labeling

proportion keywords 를 이용하면 cluster centroids 와 vocabulary index 를 이용하여 각 클러스터 별 cluster label 을 찾을 수 있습니다.

각 cluster 별로 weight 가 높은 candidates_topk 개의 후보 중에서 proportion keyword score 가 높은 topk 개의 단어가 cluster labels 로 선택됩니다. 

Proportion keywords 의 원리는 아래의 블로그에 설명되어 있습니다.

https://lovit.github.io/nlp/machine%20learning/2018/03/21/kmeans_cluster_labeling/

In [9]:
from soyclustering import proportion_keywords

keywords = proportion_keywords(
    kmeans.cluster_centers_,
    labels,
    index2word=idx_to_vocab,
    topk=30,
    candidates_topk=100
)

/Users/hyunjoongkim/git/clustering4docs/soyclustering/_keyword.py:41: RuntimeWarning: invalid value encountered in true_divide
  l1_normalize = lambda x:x/x.sum()
/Users/hyunjoongkim/git/clustering4docs/soyclustering/_keyword.py:66: RuntimeWarning: invalid value encountered in greater
  indices = np.where(p_prop > 0)[0]


keywords 는 list of list of tuple 입니다. Tuple 은 (단어, 키워드 점수)로 이뤄져 있습니다.

In [17]:
keywords[127]

[('너무너무너무', 0.9998542602089802),
 ('빅브레인', 0.9997754653094559),
 ('오블리스', 0.9996212987504849),
 ('신용재', 0.999404180820118),
 ('갓세븐', 0.9993759832053745),
 ('아이오아이', 0.9991545394514176),
 ('엠카운트다운', 0.9989354590115991),
 ('박진영', 0.9983202737395516),
 ('세븐', 0.9982670962962326),
 ('완전체', 0.9982636226082798),
 ('잠깐', 0.9976216044418017),
 ('다비치', 0.9976202378090844),
 ('중독성', 0.9973287320429198),
 ('산들', 0.9966127077338193),
 ('소녀들', 0.9964566124884715),
 ('열창', 0.996365943243375),
 ('펜타곤', 0.9957832147912045),
 ('상큼', 0.994591649024635),
 ('선율', 0.9941185744125722),
 ('엠넷', 0.9938007435078331),
 ('코드', 0.993311757541221),
 ('수록곡', 0.9928370686161826),
 ('보컬', 0.9920428723905782),
 ('엑스', 0.9911090333455465),
 ('곡으로', 0.9908721861025166),
 ('타이틀곡', 0.9907523870447625),
 ('멜로디', 0.9907008583247073),
 ('피아노', 0.9896102301762849),
 ('에이핑크', 0.9895204965930839),
 ('생방송', 0.9889897242619313)]

In [16]:
for cluster_idx, keyword in enumerate(keywords):
    if cluster_idx % 5 == 0:
        print()
    keyword = ' '.join([w for w,_ in keyword])
    print('cluster#{} : {}'.format(cluster_idx, keyword))


cluster#0 : 사업비 사업 부지 스타트업 조달 3개 민간 물류 건립 추진 지연 공사 유치 투입 시설 조성 구축 예산 담당 관광 연계 공모 도시 내년 주민 제안 개발 협력 핵심 인천
cluster#1 : 19 2015 1950 182 186 191 178 2016 600 16 196 121 500 113 179 155 90 116 194 23 152 09 72 기사 06 204 144 000 53 105
cluster#2 : 평창동계올림픽 2가지 착용감 2018 네이비 티셔츠 평창 스트레 모자 국민적 자원봉사자 맨투맨 그레이 로고 안쪽 쾌적 색상 보온성 국가대표 해피 7만 트럭 보강 의류 셔츠 편안 신발 화이트 컬러 공식
cluster#3 : 노인 참전 노인들 저소득 복지 20만원 어린이집 복지부 할머니 고령화 어르신들 사회복지 어르신 유공자 중학생 20만 예산 책정 기념식 편성 장애인 서민 부산시 인천시 행복 상담 대폭 거주 지급 인구
cluster#4 : 방류 14년 복원 자취 서식 현수막 하천 인공 금강 성숙 포획 운영할 무사 목격 중순 유입 2000 오염 9년 차바 울산시 아래 개관 춘천 국비 어린 설치 수정 자연 안내

cluster#5 : 이스라엘 로마 200 서쪽 예수 바위 그리스 성지 사원 203 기독교 기도 이슬람 600 109 111 19년 900 황금 신약 추모 105 1950 66 92 1980 개척 201 27 15
cluster#6 : 
cluster#7 : 
cluster#8 : 35 040 202 세트 345 스페셜 승리 222 106 195 점수 55 31 86 여섯 1950 68 34 28 우세 398 2018 350 95 남녀 38 82 스포츠경향 77 900
cluster#9 : 바보 음악감독 장면들 극적 보이스 트랙 히트 제아 제이 아티스트들 막강 올가을 프로듀서 녹음 특급 라인업 독점 곡으로 솔로 제작사 방영 티저 우리집 자정 속으로 캐스팅 않았던 작사 이지 입술

cluster#10 : 
cluster#11 : 